# Automated Essay Scoring

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [2]:
train = pd.read_csv("Data/train.csv")
test = pd.read_csv("Data/test.csv")

In [3]:
train.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [4]:
train.describe()

,score
count,17307.000000
mean,2.948402
std,1.044899
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,6.000000


In [5]:
from sklearn.preprocessing import OneHotEncoder

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the essay column
X = vectorizer.fit_transform(pd.concat([train['full_text'], test['full_text']], axis=0))

X = X.toarray()

# # Convert the transformed data into a DataFrame
# essay_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# # Concatenate the essay_id column with the transformed DataFrame
# essay_df = pd.concat([train['essay_id'], essay_df], axis=1)

## Neural Network Approach

In [17]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [7]:
# X = essay_df.drop("essay_id", axis=1)
y = train.score

In [8]:
import tensorflow as tf

In [18]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.L2(0.03)),
        tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal", kernel_regularizer=tf.keras.regularizers.L2(0.03)),
        tf.keras.layers.Dense(6, activation="softmax")
    ]
)

In [19]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer=tf.keras.optimizers.AdamW(weight_decay=1e-5, learning_rate=0.001,
                                      beta_1=0.9, beta_2=0.999),
    metrics="accuracy"
)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=train.score)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

In [12]:
# from scipy.sparse import csr_matrix

# # Convert sparse matrix to CSR format
# X_train = csr_matrix(X_train)
# X_test = csr_matrix(X_test)
# X_valid = csr_matrix(X_valid)

In [20]:
model.fit(
    X_train[:5000], y_train[:5000] - 1, epochs=50,
    validation_data=(X_valid, y_valid - 1),
    callbacks=[early_stopping_cb]
)

Epoch 1/50
157/157 [==============================] - 13s 80ms/step - loss: 3.2415 - accuracy: 0.3952 - val_loss: 2.0989 - val_accuracy: 0.4438
Epoch 2/50
157/157 [==============================] - 12s 76ms/step - loss: 1.7462 - accuracy: 0.5002 - val_loss: 1.5662 - val_accuracy: 0.4950
Epoch 3/50
157/157 [==============================] - 12s 75ms/step - loss: 1.4184 - accuracy: 0.5390 - val_loss: 1.3674 - val_accuracy: 0.5443
Epoch 4/50
157/157 [==============================] - 12s 76ms/step - loss: 1.3054 - accuracy: 0.5576 - val_loss: 1.3503 - val_accuracy: 0.5347
Epoch 5/50
157/157 [==============================] - 12s 77ms/step - loss: 1.2399 - accuracy: 0.5906 - val_loss: 1.2850 - val_accuracy: 0.5505
Epoch 6/50
157/157 [==============================] - 12s 79ms/step - loss: 1.2074 - accuracy: 0.5906 - val_loss: 1.2308 - val_accuracy: 0.5905
Epoch 7/50
157/157 [==============================] - 12s 79ms/step - loss: 1.1619 - accuracy: 0.6090 - val_loss: 1.2100 - val_accuracy:

In [22]:
model.evaluate(X_test, y_test - 1)

82/82 [==============================] - 1s 9ms/step - loss: 1.1417 - accuracy: 0.6007


[1.1416622400283813, 0.6006931066513062]

## Predict the values

In [39]:
pred = model.predict(X[-3:])

1/1 [==============================] - 0s 33ms/step


In [40]:
pred = np.argmax(pred, axis=1)

In [41]:
pred.shape

(3,)

In [42]:
test_X.shape

(3, 467)

In [43]:
pred

array([2, 2, 3], dtype=int64)

In [52]:
sub_df = pd.DataFrame(test.essay_id)
sub_df["score"] = pred + 1

In [51]:
sub_df.head()

,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4


In [ ]:
prediction.to_csv('submission.csv', index=False)